In [1]:
import pandas as pd

In [2]:
# We import the datasests 
dfPlay = pd.read_csv('../datasets/play_time.csv')
dfGameGenre= pd.read_csv('../datasets/game_genres.csv')
dfnameGame = pd.read_csv('../datasets/name_games.csv')
dfGenre = pd.read_csv('../datasets/genres.csv')
dfReviews = pd.read_csv('../datasets/reviews.csv')

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género. </br>

Ejemplo de retorno: 
- {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [3]:
# Merging DataFrames to combine playtime data with id_game and id_genres

playTimeGenre = pd.merge(dfPlay,dfnameGame, on='id_game', how='left')
playTimeGenre = pd.merge(playTimeGenre,dfGameGenre, on='id_game', how='left')
playTimeGenre = pd.merge(playTimeGenre, dfGenre, on='id_genre', how='left')

# Selecting specific columns and preparing the final DataFrame for analysis.
cols = ['genre', 'playtime_forever', 'anio']
playTimeGenre = playTimeGenre[cols]

playTimeGenre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12342227 entries, 0 to 12342226
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   genre             object 
 1   playtime_forever  int64  
 2   anio              float64
dtypes: float64(1), int64(1), object(1)
memory usage: 282.5+ MB


In [4]:
# Data Cleaning Operations:
# Removing rows with missing values and resetting DataFrame index
playTimeGenre.dropna(inplace=True)
playTimeGenre.reset_index(drop=True, inplace=True)
playTimeGenre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11483220 entries, 0 to 11483219
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   genre             object 
 1   playtime_forever  int64  
 2   anio              float64
dtypes: float64(1), int64(1), object(1)
memory usage: 262.8+ MB


In [5]:
# Filtering Data:
# Selecting rows where both 'playtime_forever' and 'anio' are non-zero.
playTimeGenre = playTimeGenre[(playTimeGenre['playtime_forever'] != 0) &
                              (playTimeGenre['anio'] != 0.0)]
playTimeGenre

,genre,playtime_forever,anio
0,Action,6,2000.0
2,Action,7,2003.0
8,Action,4733,2010.0
9,Action,1853,2004.0
10,Action,333,2005.0
...,...,...,...
11483215,Free to Play,3,2015.0
11483216,Adventure,3,2015.0
11483217,Casual,4,2016.0
11483218,Indie,4,2016.0


In [6]:
# Grouping and Aggregating Data:
# Grouping by 'genre' and 'anio', then calculating the total playtime for each genre and release year.

playTimeGenre = playTimeGenre.groupby(['genre','anio'])['playtime_forever'].sum().reset_index()
playTimeGenre['anio'] = playTimeGenre['anio'].astype(int)
playTimeGenre

,genre,anio,playtime_forever
0,1980s,2015,24057
1,1990's,2011,173548
2,1990's,2012,58466
3,2D,2012,168599
4,3D Platformer,2011,173548
...,...,...,...
530,Web Publishing,2015,348861
531,Web Publishing,2016,136
532,Web Publishing,2017,9713
533,Zombies,2015,2730


In [7]:
# Identifying the index with the maximum playtime for each genre.
idx = playTimeGenre.groupby('genre')['playtime_forever'].idxmax()

# Selecting rows with maximum playtime for each genre using the identified indices.
playTimeGenre = playTimeGenre.loc[idx]

playTimeGenre

,genre,anio,playtime_forever
0,1980s,2015,24057
1,1990's,2011,173548
3,2D,2012,168599
4,3D Platformer,2011,173548
31,Action,2012,1099231451
...,...,...,...
523,Violent,2015,1503
524,Walking Simulator,2015,2
527,Web Publishing,2012,1947406
533,Zombies,2015,2730


In [8]:
playTimeGenre[playTimeGenre['genre'] == 'e-sports']

,genre,anio,playtime_forever
534,e-sports,2014,451933


In [9]:
# Saving the DataFrame as a CSV file for consulting
# playTimeGenre.to_csv('data_render/playTimeGenre.csv', index=False)

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.</br>

Ejemplo de retorno: 
- {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, 
- "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [3]:
# Merging DataFrames by 'id_game', 'id_game'and 'id_user'
userGenre = pd.merge(dfPlay, dfGameGenre, on='id_game', how='left')
userGenre = pd.merge(userGenre, dfGenre, on='id_genre', how='left')
userGenre = pd.merge(userGenre,dfReviews, on='id_user', how='left')

# Selecting specific columns for analysis
cols = ['id_user','genre', 'year','playtime_forever']
userGenre = userGenre[cols]

In [4]:
# Dropping rows with missing values
userGenre.dropna(inplace=True)

# Removing rows where playtime is zero, ensuring clean and meaningful data for analysis.
userGenre = userGenre[userGenre['playtime_forever'] != 0]

In [5]:
userGenre['year'] = userGenre['year'].astype(int)
userGenre.reset_index(drop=True, inplace=True)

In [6]:
# Limiting the DataFrame to save space
userGenre = userGenre.head(1450000)

In [7]:
#Saving the DataFrame as a CSV file for consulting
#userGenre.to_csv('data_render/userGenre.csv', index=False)

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales) </br>

Ejemplo de retorno:
 - [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [8]:
# Filter rows where 'recommend' is True and 'sentiment' is positive/neutral for the given year.
usersRecommend = dfReviews[(dfReviews['recommend'] == True) &
                              (dfReviews['sentiment'].isin(['Positive', 'Neutral']))]

# Merge filtered data with additional game name information.
usersRecommend = pd.merge(usersRecommend, dfnameGame, on='id_game', how='left')

In [9]:
# Selecting specific columns for analysis and dropping rows
cols = ['name_game', 'year']
usersRecommend = usersRecommend[cols]
usersRecommend.dropna(inplace=True)

In [10]:
# Grouping by 'name_game' and 'year', counting recommendations for each game in the given year.
top_games = usersRecommend.groupby(['name_game', 'year']).size().reset_index(name='recommend_count')

In [11]:
# Saving the DataFrame as a CSV file for consulting
top_games.to_csv('data_render/usersRecommend.csv', index=False)

def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos) </br>

Ejemplo de retorno: 
- [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [12]:
# Filter rows where 'recommend' is False and 'sentiment' is negative for the given year.
usersNotRecommend = dfReviews[(dfReviews['recommend'] == False) &
                              (dfReviews['sentiment'].isin(['Negative']))]

usersNotRecommend = pd.merge(usersNotRecommend, dfnameGame, on='id_game', how='left')

In [13]:
# Selecting specific columns for analysis and dropping rows
cols = ['name_game', 'year']
usersNotRecommend = usersNotRecommend[cols]
usersNotRecommend.dropna(inplace=True)

In [14]:
# Grouping by 'name_game' and 'year', counting the worst recommendations for each game in the given year.
worst_games = usersNotRecommend.groupby(['name_game', 'year']).size().reset_index(name='recommend_count')

In [15]:
# Saving the DataFrame as a CSV file for consulting
#worst_games.to_csv('data_render/usersNotRecommend.csv', index=False)

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.</br>

Ejemplo de retorno: 

- {Negative = 182, Neutral = 120, Positive = 278}

In [16]:
# Merging 'dfReviews' DataFrame with 'dfnameGame' DataFrame based on 'id_game' using a left join.
sentAnalysis = pd.merge(dfReviews,dfnameGame, on='id_game', how ='left')

# Selecting specific columns for sentiment analysis.
# 'anio': release year
cols = ['anio', 'sentiment']
sentAnalysis = sentAnalysis[cols]

In [17]:
sentAnalysis.dropna(inplace=True)
sentAnalysis['anio'] = sentAnalysis['anio'].astype(int)

# Renaming the 'anio' column to 'release year' for clarity.
sentAnalysis.rename(columns={'anio':'release year'}, inplace=True)

In [18]:
# Grouping 'sentAnalysis' DataFrame by 'release year' and 'sentiment', counting occurrences of each sentiment.
sentAnalysis = sentAnalysis.groupby(['release year','sentiment']).size()\
    .reset_index(name='recount')

In [19]:
# Saving the DataFrame as a CSV file for consulting
#sentAnalysis.to_csv('data_render/sent_analysis.csv', index=False)